CS524: Introduction to Optimization Lecture 26
======================================

## November 1, 2024

## Piecewise Linear Cost Functions

We can use binary variables to model arbitrary piecewise linear functions.

The function is nonconvex in general, and is specified by ordered pairs $(B_i,f(B_i))$.

Intuitively, one can consider the endpoints of segments as "breakpoints" and the segments themselves as "pieces," with there being one more breakpoint than piece.

<img src="./images/piecewise-linear.png" width="500">


In [1]:
import sys
import numpy as np

from gamspy import Container, Problem, Sense, Sum, Smin, Domain, Card, Options, Number, Ord
import gamspy.math as gpm

## Purchase Example

**FEATURES**:

MIP problem, modeling a piecewise linear function with binaries, graphical representation of data

**DESCRIPTION:**  

There are three suppliers of a good, and they have quoted
                 various prices for various quantities of product. We want
                 to buy at least total cost, yet not buy too much from any
                 one supplier. Each supplier offers decreasing prices for
                 increased lot size, in the form of incremental discounts.
                 We wish to buy 600 items in total.

* Note that this model has a non-zero fixed cost that is applied for all suppliers
* The purchase model doesn't apply this if the unit is not used



## Definition of Sets and Parameters

- $S$: Set of suppliers
- $I$: Set of cost breakpoints
- $B_{si}$: Maximum number of items from supplier $s \in S$ to purchase at breakpoint $i \in I$
- $CBR_{si}$: Cost of item from supplier $s \in S$ at breakpoint $i \in I$
- $R$: Number of purchase
- $\alpha_s$: Maximum percentage to purchase from any supplier

In [2]:
class arguments:
  def __init__(self, big=False, fcost=False, solver='cplex', sos1='binary'):
    self.big = big
    self.fcost = fcost
    self.solver = solver
    self.sos1 = sos1
    self.NB = 480
    self.NS = 140

args = arguments(big=True,fcost=True)

options = Options(relative_optimality_gap=1e-6,threads=3,seed=666)
cont = Container(options=options)

if args.big:
    i = cont.addSet('i',description='price break points',records=range(0,args.NB+1))
    seg = cont.addSet('seg',domain=[i],description='price break segments',records=range(1,args.NB+1))
    s = cont.addSet('s',description='Suppliers',records=[f"s{ind+1}" for ind in range(0,args.NS)])

    CBR = cont.addParameter('CBR',domain=[s,i],description='Total cost at break points')
    B = cont.addParameter('BR',domain=[s,i],description='Breakpoints (quantities at which unit cost changes)')

    # generate random values for CBR and B
    BSIZE = cont.addParameter('BSIZE')
    BSIZE[:] = gpm.Round(gpm.uniform(1,4))
    BASECOST = cont.addParameter('BASECOST',domain=[s],description='base Unit cost')
    BASECOST[s] = gpm.uniform(10,12)
    BASEBREAK = cont.addParameter('BASEBREAK',domain=[s],description='base break level')
    BASEBREAK[s] = gpm.Round(gpm.uniform(50,150))
    # generate function values at 0, and slopes for segments
    CBR[s,i] = BASECOST[s] * (gpm.uniform(-1,1)+gpm.power(gpm.uniform(0.85,0.95), i.ord-1))
    B[s,i].where[i.ord>=2] = BASEBREAK[s]*BSIZE*(i.ord-1) + BASEBREAK[s]*BSIZE*gpm.uniform(0.9,1.1)
    for ind in seg.toList():
        # on rhs CBR[s,i] are the segment slopes
        CBR[s,i].where[i.val == ind] = CBR[s,i.lag(1)] + CBR[s,i]*(B[s,i]-B[s,i.lag(1)])
            
    ALPHA=cont.addParameter('ALPHA',domain=[s],description='Maximum percentages for each supplier')
    ALPHA[s] = 10*gpm.Round(gpm.uniform(100,250)/Card(s))/100
    R = cont.addParameter('REQ', description='Total quantity required')
    R[:] = Sum(Domain(s,i).where[i.ord==gpm.Round(Card(i)/2)], B[s,i])/Card(s)
else:
    i = cont.addSet('i',description='price break points',records=range(0,4))
    seg = cont.addSet('seg',domain=[i],description='price break segments',records=range(1,4))
    s = cont.addSet('s',description='Suppliers',records=[f"s{ind+1}" for ind in range(0,3)])

    CBR = cont.addParameter('CBR',domain=[s,i],description='Total cost at break points',records=np.array([
        [30.0000,    950.0000,   1850.0000,   7450.0000],
        [ 8.0000,    458.0000,   2158.0000,  16683.0000],
        [10.0000,   1110.0000,   2810.0000,  30560.0000 ]]) )
    B = cont.addParameter('BR',domain=[s,i],description='Breakpoints (quantities at which unit cost changes)',records=np.array([
        [0, 100, 200, 1000],
        [0,  50, 250, 2000],
        [0, 100, 300, 4000]]))

    ALPHA=cont.addParameter('ALPHA',domain=[s],description='Maximum percentages for each supplier',
        records=[('s1', 0.60), ('s2', 0.45), ('s3', 0.60)])
    R = cont.addParameter('REQ', description='Total quantity required', records=600)

Compute the slope and intercept values for all segments

In [3]:
# Convert data to needed format
m = cont.addParameter('m',domain=[s,i],description='gradient on segment')
m[s,i].where[seg[i]] = (CBR[s,i]-CBR[s,i.lag(1)])/(B[s,i]-B[s,i.lag(1)])

c = cont.addParameter('c',domain=[s,i],description='intercept cost on segment')
c[s,i].where[seg[i]] = CBR[s,i.lag(1)] - m[s,i]*B[s,i.lag(1)]

Modeling the constraints in GAMSPy:

In [4]:
# MODEL

x = cont.addVariable('x','positive',domain=[s],description='Quantity to purchase from supplier s')
b = cont.addVariable('b',args.sos1,domain=[s,i],description='use piece for supplier')
z = cont.addVariable('z','binary',domain=[s],description='use supplier s')
w = cont.addVariable('w','free',domain=[s,i],description= 'x values in segment k for supplier s')

# Define buy and also order the weight variables by breakpoint quantities
defbuy = cont.addEquation('defbuy',domain=[s])
defbuy[s]= Sum(seg[i], w[s,i]) == x[s]

wlo = cont.addEquation('wlo',domain=[s,i])
wlo[s,seg[i]]= B[s,i.lag(1)]*b[s,i] <= w[s,i]

wup = cont.addEquation('wup',domain=[s,i])
wup[s,seg[i]]= w[s,i] <= B[s,i]*b[s,i]

# The convexity row (b sum to 1)
OnePiece = cont.addEquation('OnePiece',domain=[s])
if args.fcost:
    OnePiece[s]= Sum(seg[i], b[s,i]) == z[s]
else:
    OnePiece[s]= Sum(seg[i], b[s,i]) == 1

# The minimum quantity that must be bought
Demand = cont.addEquation('Demand')
Demand[:] = Sum(s, x[s]) >= R

purchase = cont.addModel('purchase',
    equations=cont.getEquations(),
    problem=Problem.MIP,
    sense=Sense.MIN,
    objective=Sum([s,seg[i]], c[s,i]*b[s,i] + m[s,i]*w[s,i])
)

# No more than the maximum percentage from each supplier
x.up[s] = ALPHA[s]*R

purchase.solve(solver=args.solver,output=None)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,14109.2889431449,134682,134681,MIP,CPLEX,0.212


In [5]:
print(f"Cost = {purchase.objective_value}, time = {purchase.total_solver_time}")
print(f"Use segment =\n {b.pivot()}\nw =\n {w.pivot()}")
if args.fcost:
    print(f"Use supplier =\n {z.l.records}")

Cost = 14109.288943144893, time = 0.3210000693798065
Use segment =
         1    2    3    4    5    6    7    8    9   10  ...  471  472  473  \
s1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s5    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
s136  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s137  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s138  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s139  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s140  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0

## Infinite domains

We explain this approximation for a function $f$ of a single variable $x$. The piecewise-linear function is described by a collection of segments $\mathcal{S}$. In the case where the domain of the function is an unbounded set, or the function is not continuous, the segment approach has proven effective. Each segment $i$ has an $(B_i,f_i)$ coordinate point, a (potentially infinite) length $l_i$, and a slope $m_i$, the rate of increase or decrease of the function from $(B_i,f_i)$.

<img src="images/piecewise-linear.png" width="500">

The sign of the $l_i$ determines if the segment expands to the left (negative length) or the right (positive length) of the $(B_i,f_i)$ point. These segment definitions allow more than pure piecewise-linear functions.  Segments can overlap, meaning we can have multi-valued functions, and there can be holes in the $x$ coordinate space. There is also no order requirement of the segment $B_i$ coordinates.

Each segment has two variables associated with it. The first is a binary
variable $b_i$ that chooses the segment to be used. In order that we have a single value for the function at $x$, only one segment can be active, which is modeled using:
$$\sum_{i \in \mathcal{S}} b_i = 1.$$
The other segment variable is a nonnegative variable $\lambda_i$ whose upper bound is the absolute value of the length of the segment: $\lambda_i \leq |l_i|$. This variable measures how far we move into segment $i$ from the starting point $(B_i,f_i)$ A particular choice of the vectors $b$ and $\lambda$ formed from these components determines a point of evaluation $x \in \mathbb{R}$ and the value of the approximation $f$ at $x$ by the following formulae ($\text{sgn}(l_i)$ denotes the “sign” of the parameter $l_i$):
$$x = \sum_{i \in \mathcal{S}}(B_ib_i + \text{sgn}(l_i)\lambda_i), f = \sum_{i \in \mathcal{S}}(f_ib_i + \text{sgn}(l_i)m_i\lambda_i)$$

## Infinite Domains

For each segment that has finite length $|l_i| < \infty$, we enforce the constraint that $\lambda_i$ can only be positive if $b_i = 1$ using the $M$ constraint:
$$\lambda_i \leq |l_i|b_i.$$
If the piecewise-linear function contains segments of infinite length, this constraint does not work.  Instead, for these segments, we form a SOS1 set containing the variables $\lambda_i$ and $1-b_i$, that is at most one of these two variables is positive. This has the same effect as the $M$ constraint, but is independent of the length of the segment and hence also works with infinite length. (Note: this is not needed in the purchase example)

In [6]:
plusminus = cont.addParameter('plusminus',domain=[s,i],description='plus/minus from base')
plusminus[s,i].where[seg[i]] = gpm.sign(B[s,i.lag(1)] - B[s,i])
# MODEL

x = cont.addVariable('x','positive',domain=[s],description='Quantity to purchase from supplier s')
b = cont.addVariable('b',args.sos1,domain=[s,i],description='use piece for supplier')
z = cont.addVariable('z','binary',domain=[s],description='use supplier s')
lamda = cont.addVariable('lamda','positive',domain=[s,i],description= 'length along interval')

# Define buy and also order the weight variables by breakpoint quantities
defbuy = cont.addEquation('defbuy',domain=[s])
defbuy[s] = Sum(seg[i], b[s,i]*B[s,i] + plusminus[s,i]*lamda[s,i]) == x[s]

killOff = cont.addEquation('killOff',domain=[s,i])
killOff[s,seg[i]] = lamda[s,i] <= gpm.abs(B[s,i]-B[s,i.lag(1)])*b[s,i]

# The convexity row (b sum to 1)
OnePiece = cont.addEquation('OnePiece',domain=[s])
if args.fcost:
    OnePiece[s] = Sum(seg[i], b[s,i]) == z[s]
else:
    OnePiece[s] = Sum(seg[i], b[s,i]) == 1

# The minimum quantity that must be bought
Demand = cont.addEquation('Demand')
Demand[:] = Sum(s, x[s]) >= R

purchase = cont.addModel('purchase',
    equations=cont.getEquations(),
    problem=Problem.MIP,
    sense=Sense.MIN,
    objective=Sum([s,seg[i]], CBR[s,i]*b[s,i] + plusminus[s,i]*m[s,i]*lamda[s,i])
)

# No more than the maximum percentage from each supplier
x.up[s] = ALPHA[s]*R

purchase.solve(solver=args.solver,output=None)

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,14109.2889431449,201882,201881,MIP,CPLEX,0.585


In [10]:
print(f"Cost = {purchase.objective_value}, time = {purchase.total_solver_time}")
print(f"Use segment =\n {b.pivot()}\nlamda =\n {lamda.pivot()}")
if args.fcost:
    print(f"Use supplier =\n {z.l.records}")

Cost = 14109.288943144908, time = 0.7410000544041395
Use segment =
         1    2    3    4    5    6    7    8    9   10  ...  471  472  473  \
s1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s3    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s5    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
s136  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s137  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s138  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s139  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
s140  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0

## Further Information

For further information, refer to the slides using the following link:

https://canvas.wisc.edu/courses/426147/files/folder/Lectures?preview=42088737